In [ ]:
from trustutils import run
#from trustutils import files
#from trustutils import plot
#import os, sys
#import numpy as np
#from matplotlib import pyplot
#import math

run.introduction("J. Francescatto","17/02/2025")
run.TRUST_parameters()


## Description du Probleme:


Le test décrit ci-dessous vise à vérifier la chute de gouttes de liquide de diamètres variés dans un gaz, sous l’effet de la gravité et du frottement interfacial. Dans cette simulation, les termes de frottement spécifiques à chaque phase sont négligés.

Le domaine de calcul en 2D mesure 4,5 m de large et 5 m de haut. Le maillage régulier est composé de 451 x 501 nœuds, soit 10 mailles par cellule pour une taille de 0,1 m. Ces fluides respectent la loi d'état Stiffened Gas, avec les paramètres suivants :
1. **Liquide :**
   \begin{equation}
   P_{\infty}= 8,5 \text{ } 10^{8} \text{ Pa, } \quad \gamma = 2.8 \quad \text{et} \quad c_p = 1462 \, \text{J/kg/K}
   \end{equation}

2. **Gaz :**
   \begin{equation}
   P_{\infty}= 0 \text{ Pa, } \quad \gamma = 1.4 \quad \text{et} \quad c_p = 717 \, \text{J/kg/K}
   \end{equation}


La description de la phase dispersée dans le jeu de données est le suivant :

- Un fluide porteur, composé de gaz parfait, nommé « gaz_continu_air » ;
- Un fluide dispersé de gouttes de liquide dont le diamètre varie entre $dmin=5\text{ } 10^{-5} m$ et $dmax=5\text{ } 10^{-2} m$. Ce fluide est discrétisé en trois 
    classes de tailles de gouttes et est désigné sous le nom de « liquide_dispersee_eau ».
    
 
Le calcul des bornes des trois classes suit une répartition logarithmique :

- Classe n°1 : contient de petites gouttes de liquide, avec un diamètre de Sauter d’environ $1,95\text{ } 10^{-4}$;
- Classe n°2 : contient des gouttes de taille moyenne, avec un diamètre de Sauter d’environ $1,95\text{ } 10^{-3}$;
- Classe n°3 : contient de grosses gouttes de liquide, avec un diamètre de Sauter d’environ $1,95\text{ } 10^{-2}$.


Pour chaque classe de dispersion ainsi que pour la phase porteuse, les équations de conservation de la masse, de la quantité de mouvement et de l’énergie interne sont résolues, ce qui conduit à un total de 16 équations à discrétiser et à résoudre.

Initialement, les fluides sont au repos, avec une température uniforme de 20 °C. Les densités sont de $\rho_l=10^{3} kg.m{-3}$ pour le liquide et de $\rho_g=1 kg.m{-3}$ pour le gaz. Trois zones contenant des gouttes sont définies, chacune correspondant à une classe de particules spécifique. Cela permet d’analyser de manière distincte l’effet de chaque classe et de vérifier que chaque classe contient bien des particules de tailles différentes. La définition des fractions volumiques pour chaque zone est la suivante :

- Phase porteuse (gaz) :  $\alpha_g=1$ sauf dans le domaine des carrées où $\alpha_g=0.9$ ;
- Phase dispersée (liquide) est divisée en trois classes de tailles de gouttes représentées dans chaque carrée :
  - Classe N° 1 : fraction volumique de $\alpha_l^{1}=0.1$ dans le carré de gauche et $\alpha_l^{1}=0$ en dehors ;
  - Classe N° 2 : fraction volumique de $\alpha_l^{2}=0.1$ dans le carré du milieu et $\alpha_l^{2}=0$ en dehors ;
  - Classe N° 3 : fraction volumique de $\alpha_l^{3}=0.1$ dans le carré de droite et $\alpha_l^{3}=0$ en dehors.
  
Les conditions aux limites sont définies comme suit : les côtés et la partie inférieure du domaine sont des parois fixes, tandis que la partie supérieure est ouverte à la pression atmosphérique. La discrétisation spatiale est réalisée en Volumes Différences Finies (VDF), et la discrétisation temporelle utilise le schéma d’Euler implicite du module « Pb_multiphase ».







In [ ]:
force_recalculation = True
build_dir = run.BUILD_DIRECTORY

Coeff = 440.0 
dis_list = ['VDF'] #liste des discrétisations

if force_recalculation or not os.path.exists(f'{build_dir}'):
    print('Recalculation running..')
    run.reset()
    
    substitutions_dict = {
        "Coeff" : Coeff, 
        "Nx": 227,
        "Ny": 251
    }

    # Use a smaller mesh for NR tests:
    if run.isExtractingNR():
        substitutions_dict["Nx"] = 51
        substitutions_dict["Ny"] = 51
        
    for dis in dis_list:
        r = run.addCaseFromTemplate(f"jdd.data", targetDirectory=f"{dis}", dic=substitutions_dict, nbProcs=16)
        r.partition()
            
    run.printCases()
    run.runCases()
    run.tablePerf()

else:
    print('No recalculation, reusing old results!')


In [ ]:
from trustutils import visit

def annotations():
  a.visitCommand("annotations = AnnotationAttributes()")
  a.visitCommand("annotations.axes2D.xAxis.title.visible = 0")
  a.visitCommand("annotations.axes2D.yAxis.title.visible = 0")
  a.visitCommand("annotations.userInfoFlag = 0")
  a.visitCommand("annotations.databaseInfoFlag = 0")
  a.visitCommand("annotations.axes2D.xAxis.grid = 0")
  a.visitCommand("annotations.axes2D.yAxis.grid = 0")
  a.visitCommand('slider = CreateAnnotationObject("TimeSlider")')
  a.visitCommand("slider.position = (0.3, 0.05)")
  a.visitCommand("SetAnnotationAttributes(annotations)")
    
def titre():
  a.visitCommand('banner = CreateAnnotationObject("Text2D")')
  a.visitCommand('banner.text = "Cas test de verification - MUSIG"')
  a.visitCommand("banner.position = (0.3, 0.95)")
  a.visitCommand("banner.height = 0.02")
  a.visitCommand("banner.fontBold = 1")

def cadre():
  a.visitCommand('line1 = CreateAnnotationObject("Line2D")')
  a.visitCommand("line1.width = 5")
  a.visitCommand("line1.position = (0.199, 0.15)")
  a.visitCommand("line1.position2 = (0.92, 0.15)")
  a.visitCommand('line2 = CreateAnnotationObject("Line2D")')
  a.visitCommand("line2.width = 5")
  a.visitCommand("line2.position = (0.2, 0.15)")
  a.visitCommand("line2.position2 = (0.2, 0.95)")
  a.visitCommand('line3 = CreateAnnotationObject("Line2D")')
  a.visitCommand("line3.width = 5")
  a.visitCommand("line3.position = (0.92, 0.15)")
  a.visitCommand("line3.position2 = (0.92, 0.95)")
    
def legend():
  a.visitCommand("legend = GetAnnotationObject(GetPlotList().GetPlots(0).plotName)")
  a.visitCommand("legend.fontBold = 1")
  a.visitCommand("legend.useCustomTitle = 1")
  a.visitCommand('legend.customTitle = "Fraction de Gaz"')

a = visit.Show("VDF/PAR_jdd.lata", "Pseudocolor","ALPHA_GAZ_CONTINU_AIR_ELEM_dom",plotmesh=False)
annotations()
titre()
cadre()
legend()
a.plot()

